In [ ]:
# Import system modules
import os
import sys
import arcpy
from arcpy import management as management
from arcpy import analysis as analysis

arcpy.env.overwriteOutput = True

# Set local variables
input_lakes = input('Insira a camada de lago:')
input_water_bodies = input('Insira a camada de corpo dagua:')
input_rivers = input('Insira a camada de hidrografia:')
input_properties = input('Insira a camada de propriedades:')
input_springs = input('Insira a camada de nascente:')
input_veg_2008 = input('Insira a camada de vegetacao 2008:')
input_veg_current = input('Insira a camada de vegetacao atual:')
input_ppa_areas = input('Insira a camada de PPA:')
input_saf = input('Insira a camada de SAF:')
input_others_uses = input('Insira a camada de outras áreas:')
output = input('Insira endereco de output:')
input_toolbox_path = input('Insira endereco do toolbox:')


In [ ]:
# funcao responsavel por copiar os arquivos de input para o dataset de input do FGDB criado
def create_fgdb(v_output):
    fgdb_name = 'forest_code_analisys.gdb'
    fgdb = management.CreateFileGDB(out_folder_path=v_output, out_name=fgdb_name, out_version='CURRENT')  
    dataset_input = management.CreateFeatureDataset(out_dataset_path=fgdb, out_name='input', spatial_reference=input_properties)
    dataset_intermediate_results = management.CreateFeatureDataset(out_dataset_path=fgdb, out_name='intermediate_results', spatial_reference=input_properties)
    dataset_intermediate_results_outputs = management.CreateFeatureDataset(out_dataset_path=fgdb, out_name='intermediate_results_outputs', spatial_reference=input_properties)
    dataset_outputs = management.CreateFeatureDataset(out_dataset_path=fgdb, out_name='outputs', spatial_reference=input_properties)
    dataset_outputs_calculate = management.CreateFeatureDataset(out_dataset_path=fgdb, out_name='outputs_calculate', spatial_reference=input_properties)

    return dataset_input, dataset_intermediate_results, dataset_intermediate_results_outputs, dataset_outputs, dataset_outputs_calculate, fgdb

In [ ]:
# funcao responsavel por copiar os arquivos de input para o dataset de input do FGDB criado
def add_input(inp_lakes, inp_water_bodies, inp_rivers, inp_properties, inp_springs, inp_veg_2008, inp_veg_current, inp_ppa_areas, inp_saf, inp_others_uses, output, inp_toolbox_path):
    nm_lakes = f'{output}\lakes'
    lakes = management.CopyFeatures(in_features=inp_lakes, out_feature_class= nm_lakes)
    nm_water_bodies = f'{output}\water_bodies'
    water_bodies = management.CopyFeatures(in_features=inp_water_bodies, out_feature_class= nm_water_bodies)
    nm_rivers = rf'{output}\rivers'
    rivers = management.CopyFeatures(in_features=inp_rivers, out_feature_class= nm_rivers)
    nm_properties = f'{output}\properties'
    properties = management.CopyFeatures(in_features=inp_properties, out_feature_class=nm_properties)
    nm_springs = f'{output}\springs'
    springs = management.CopyFeatures(in_features=inp_springs, out_feature_class=nm_springs)
    nm_veg_2008 = rf'{output}\veg_2008'
    veg_2008 = management.CopyFeatures(in_features=inp_veg_2008, out_feature_class=nm_veg_2008)
    nm_veg_current = rf'{output}\veg_current'
    veg_current = management.CopyFeatures(in_features=inp_veg_current, out_feature_class=nm_veg_current)
    nm_ppa_areas = f'{output}\ppa_areas'
    if input_ppa_areas != "":
        ppa_areas = management.CopyFeatures(in_features=inp_ppa_areas, out_feature_class=nm_ppa_areas)
    nm_saf = rf'{output}\saf'
    if input_saf != "":
        saf = management.CopyFeatures(in_features=inp_saf, out_feature_class=nm_saf)
    nm_others_uses = rf'{output}\others_uses'
    if input_others_uses != "":
        others_uses = management.CopyFeatures(in_features=inp_others_uses, out_feature_class=nm_others_uses)
    nm_city = f'{output}\municipio'
    city_path = os.path.join(inp_toolbox_path, r'Project\forest_code_analisys.gdb\input', 'municipio')
    city = management.CopyFeatures(in_features=city_path, out_feature_class= nm_city)
    nm_uf = fr'{output}\uf'
    uf_path = os.path.join(inp_toolbox_path, r'Project\forest_code_analisys.gdb\input', 'uf')
    uf = management.CopyFeatures(in_features=uf_path, out_feature_class= nm_uf)

    return nm_lakes, nm_water_bodies, nm_rivers, nm_properties, nm_springs, nm_veg_2008, nm_veg_current, nm_ppa_areas, nm_saf, nm_others_uses, nm_city, nm_uf

In [ ]:
# funcao responsavel por criar os arquivos temporarios do modulo config do antigo Model Builder
def config(v_lakes, v_water_bodies, v_rivers, v_properties, v_output):

    # Process: Make Feature Layer (5) (Make Feature Layer) (management)
    lakes_layer = management.MakeFeatureLayer(in_features=v_lakes, out_layer='lyr_lakes', where_clause='', workspace=v_output)

    # Process: Select Layer By Attribute (5) (Select Layer By Attribute) (management)
    lakes_layer_t2 = management.SelectLayerByAttribute(in_layer_or_view=lakes_layer, selection_type='NEW_SELECTION', where_clause='area >=1 AND area <=20')

    # Process: Copy Features (6) (Copy Features) (management)
    lagoa_area_1_20 = f'{v_output}\lagoa_area_1_20'
    management.CopyFeatures(in_features=lakes_layer_t2, out_feature_class=lagoa_area_1_20)

    # Process: Select Layer By Attribute (6) (Select Layer By Attribute) (management)
    lakes_layer_sla1 = management.SelectLayerByAttribute(in_layer_or_view=lakes_layer, selection_type='NEW_SELECTION', where_clause='area >20')

    # Process: Copy Features (7) (Copy Features) (management)
    lagoa_area_maior_20 = f'{v_output}\lagoa_area_maior_20'
    management.CopyFeatures(in_features=lakes_layer_sla1, out_feature_class=lagoa_area_maior_20)

    # Process: Merge (Merge) (management)
    pol_erase = f'{v_output}\pol_erase'
    management.Merge(inputs=[v_water_bodies, v_lakes], output=pol_erase) # avaliar se nao tem que colocar o field_mappings

    # Process: Select Layer By Attribute (7) (Select Layer By Attribute) (management)
    lakes_layer_sla2 = management.SelectLayerByAttribute(in_layer_or_view=lakes_layer, selection_type='NEW_SELECTION', where_clause='area >=1')

    # Process: Copy Features (11) (Copy Features) (management)
    lagoa_area_maior_1 = f'{v_output}\lagoa_area_maior_1'
    management.CopyFeatures(in_features=lakes_layer_sla2, out_feature_class=lagoa_area_maior_1)

    # Process: Make Feature Layer (8) (Make Feature Layer) (management)
    lagoa_area_maior_1_layer = management.MakeFeatureLayer(in_features=lagoa_area_maior_1, out_layer='lagoa_area_maior_1_layer', where_clause='', workspace=v_output) #sera que precisa disso? Quem sabe 'lagoa_area_maior_1' ja resolve

    # Process: Select Layer By Location (4) (Select Layer By Location) (management)
    lagoa_area_maior_1_sll2 = management.SelectLayerByLocation(in_layer=lagoa_area_maior_1_layer, overlap_type='INTERSECT', select_features=v_rivers, search_distance='', selection_type='NEW_SELECTION', invert_spatial_relationship='NOT_INVERT')

    # Process: Copy Features (8) (Copy Features) (management)
    lagoas_2_4mf = f'{v_output}\lagoas_2_4mf'
    management.CopyFeatures(in_features=lagoa_area_maior_1_sll2, out_feature_class=lagoas_2_4mf)

    # Process: Make Feature Layer (Make Feature Layer) (management)
    properties_layer = management.MakeFeatureLayer(in_features=v_properties, out_layer='properties_layer', where_clause='', workspace= v_output)

    # Process: Select Layer By Attribute (3) (Select Layer By Attribute) (management)
    properties_layer_sla1 = management.SelectLayerByAttribute(in_layer_or_view=properties_layer, selection_type='NEW_SELECTION', where_clause='mf <=1', invert_where_clause='')

    # Process: Copy Features (2) (Copy Features) (management)
    imovel_0_1mf = f'{v_output}\imovel_0_1mf'
    management.CopyFeatures(in_features=properties_layer_sla1, out_feature_class=imovel_0_1mf)

    # Process: Select Layer By Location (2) (Select Layer By Location) (management)
    lagoa_area_maior_1_sll3 = management.SelectLayerByLocation(in_layer=lagoa_area_maior_1_layer, overlap_type='INTERSECT', select_features=imovel_0_1mf, search_distance='', selection_type='NEW_SELECTION', invert_spatial_relationship='NOT_INVERT')

    # Process: Copy Features (9) (Copy Features) (management)
    lagoas_0_1mf = f'{v_output}\lagoas_0_1mf'
    management.CopyFeatures(in_features=lagoa_area_maior_1_sll3, out_feature_class=lagoas_0_1mf)

    # Process: Select Layer By Attribute (4) (Select Layer By Attribute) (management)
    properties_layer_sla2 = management.SelectLayerByAttribute(in_layer_or_view=properties_layer, selection_type='NEW_SELECTION', where_clause='mf >4', invert_where_clause='')

    # Process: Copy Features (5) (Copy Features) (management)
    imovel_maior_4mf = f'{v_output}\imovel_maior_4mf'
    management.CopyFeatures(in_features=properties_layer_sla2, out_feature_class=imovel_maior_4mf)

    # Process: Select Layer By Location (Select Layer By Location) (management)
    lagoa_area_maior_1_sll4 = management.SelectLayerByLocation(in_layer=lagoa_area_maior_1_layer, overlap_type='INTERSECT', select_features=imovel_maior_4mf, search_distance='', selection_type='NEW_SELECTION', invert_spatial_relationship='NOT_INVERT')

    # Process: Copy Features (10) (Copy Features) (management)
    lagoas_maior_4mf = f'{v_output}\lagoas_maior_4mf'
    management.CopyFeatures(in_features=lagoa_area_maior_1_sll4, out_feature_class=lagoas_maior_4mf)

    # Process: Select Layer By Attribute (2) (Select Layer By Attribute) (management)
    properties_layer_sla3 = management.SelectLayerByAttribute(in_layer_or_view=properties_layer, selection_type='NEW_SELECTION', where_clause='mf >1 AND mf <=2', invert_where_clause='')

    # Process: Copy Features (3) (Copy Features) (management)
    imovel_1_2mf = f'{v_output}\imovel_1_2mf'
    management.CopyFeatures(in_features=properties_layer_sla3, out_feature_class=imovel_1_2mf)

    # Process: Select Layer By Location (3) (Select Layer By Location) (management)
    # era linha 99 e trouxe para ca devido a layer imovel_1_2mf
    lagoa_area_maior_1_sll1 = management.SelectLayerByLocation(in_layer=lagoa_area_maior_1_layer, overlap_type='INTERSECT', select_features=imovel_1_2mf, search_distance='', selection_type='NEW_SELECTION', invert_spatial_relationship='NOT_INVERT')

    # Process: Copy Features (Copy Features) (management)
    lagoas_1_2mf = f'{v_output}\lagoas_1_2mf'
    management.CopyFeatures(in_features=lagoa_area_maior_1_sll1, out_feature_class=lagoas_1_2mf)

    # Process: Clip (Clip) (analysis)
    rivers_1_2mf = rf'{v_output}\rivers_1_2mf'
    analysis.Clip(in_features=v_rivers, clip_features=imovel_1_2mf, out_feature_class=rivers_1_2mf, cluster_tolerance='')

    # Process: Select Layer By Attribute (Select Layer By Attribute) (management)
    properties_layer_sla4 = management.SelectLayerByAttribute(in_layer_or_view=properties_layer, selection_type='NEW_SELECTION', where_clause='mf >2 AND mf <=4', invert_where_clause='')

    # Process: Copy Features (4) (Copy Features) (management)
    imovel_2_4mf = f'{v_output}\imovel_2_4mf'
    management.CopyFeatures(in_features=properties_layer_sla4, out_feature_class=imovel_2_4mf)

    # Process: Clip (2) (Clip) (analysis)
    rivers_2_4mf = rf'{v_output}\rivers_2_4mf'
    analysis.Clip(in_features=v_rivers, clip_features=imovel_2_4mf, out_feature_class=rivers_2_4mf, cluster_tolerance='')

    # Process: Clip (3) (Clip) (analysis)
    rivers_0_1mf = rf'{v_output}\rivers_0_1mf'
    analysis.Clip(in_features=v_rivers, clip_features=imovel_0_1mf, out_feature_class=rivers_0_1mf, cluster_tolerance='')

    # Process: Clip (4) (Clip) (analysis)
    rivers_maior_4mf = rf'{v_output}\rivers_maior_4mf'
    analysis.Clip(in_features=v_rivers, clip_features=imovel_maior_4mf, out_feature_class=rivers_maior_4mf, cluster_tolerance='')

    # Process: Clip (5) (Clip) (analysis)
    wb_maior_4mf = f'{v_output}\wb_maior_4mf'
    analysis.Clip(in_features=v_water_bodies, clip_features=imovel_maior_4mf, out_feature_class=wb_maior_4mf, cluster_tolerance='')

    # Process: Clip (6) (Clip) (analysis)
    wb_0_1mf = f'{v_output}\wb_0_1mf'
    analysis.Clip(in_features=v_water_bodies, clip_features=imovel_0_1mf, out_feature_class=wb_0_1mf, cluster_tolerance='')

    # Process: Clip (7) (Clip) (analysis)
    wb_1_2mf = f'{v_output}\wb_1_2mf'
    analysis.Clip(in_features=v_water_bodies, clip_features=imovel_1_2mf, out_feature_class=wb_1_2mf, cluster_tolerance='')

    # Process: Clip (8) (Clip) (analysis)
    wb_2_4mf = f'{v_output}\wb_2_4mf'
    analysis.Clip(in_features=v_water_bodies, clip_features=imovel_2_4mf, out_feature_class=wb_2_4mf, cluster_tolerance='')

In [ ]:
# funcao responsavel por criar os arquivos temporarios do modulo app_consolidada do antigo Model Builder
def app_consolidada(v_properties, v_springs, v_output):
    
    # Process: Buffer (13) (Buffer) (analysis)
    springs_benesses_buffer = f'{v_output}\springs_benesses_buffer'
    analysis.Buffer(in_features=v_springs, out_feature_class=springs_benesses_buffer, buffer_distance_or_field='15 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (5) (Buffer) (analysis)
    wb_maior_4mf = f'{v_output}\wb_maior_4mf'
    wb_maior_4mf_Buffer = f'{v_output}\wb_maior_4mf_Buffer'
    analysis.Buffer(in_features=wb_maior_4mf, out_feature_class=wb_maior_4mf_Buffer, buffer_distance_or_field='30 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (6) (Buffer) (analysis)
    wb_2_4mf = f'{v_output}\wb_2_4mf'
    wb_2_4mf_Buffer = f'{v_output}\wb_2_4mf_Buffer'
    analysis.Buffer(in_features=wb_2_4mf, out_feature_class=wb_2_4mf_Buffer, buffer_distance_or_field='15 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (7) (Buffer) (analysis)
    wb_1_2mf = f'{v_output}\wb_1_2mf'
    wb_1_2mf_Buffer = f'{v_output}\wb_1_2mf_Buffer'
    analysis.Buffer(in_features=wb_1_2mf, out_feature_class=wb_1_2mf_Buffer, buffer_distance_or_field='8 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (8) (Buffer) (analysis)
    wb_0_1mf = f'{v_output}\wb_0_1mf'
    wb_0_1mf_Buffer = f'{v_output}\wb_0_1mf_Buffer'
    analysis.Buffer(in_features=wb_0_1mf, out_feature_class=wb_0_1mf_Buffer, buffer_distance_or_field='5 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (4) (Buffer) (analysis)
    rivers_maior_4mf = rf'{v_output}\rivers_maior_4mf'
    rivers_maior_4mf_Buffer = rf'{v_output}\rivers_maior_4mf_Buffer'
    analysis.Buffer(in_features=rivers_maior_4mf, out_feature_class=rivers_maior_4mf_Buffer, buffer_distance_or_field='20 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (3) (Buffer) (analysis)
    rivers_2_4mf = rf'{v_output}\rivers_2_4mf'
    rivers_2_4mf_Buffer = rf'{v_output}\rivers_2_4mf_Buffer'
    analysis.Buffer(in_features=rivers_2_4mf, out_feature_class=rivers_2_4mf_Buffer, buffer_distance_or_field='15 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (2) (Buffer) (analysis)
    rivers_1_2mf = rf'{v_output}\rivers_1_2mf'
    rivers_1_2mf_Buffer = rf'{v_output}\rivers_1_2mf_Buffer'
    analysis.Buffer(in_features=rivers_1_2mf, out_feature_class=rivers_1_2mf_Buffer, buffer_distance_or_field='8 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (Buffer) (analysis)
    rivers_0_1mf = rf'{v_output}\rivers_0_1mf'
    rivers_0_1mf_Buffer = rf'{v_output}\rivers_0_1mf_Buffer'
    analysis.Buffer(in_features=rivers_0_1mf, out_feature_class=rivers_0_1mf_Buffer, buffer_distance_or_field='5 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (9) (Buffer) (analysis)
    lagoas_0_1mf = f'{v_output}\lagoas_0_1mf'
    lagoas_0_1mf_Buffer = f'{v_output}\lagoas_0_1mf_Buffer'
    analysis.Buffer(in_features=lagoas_0_1mf, out_feature_class=lagoas_0_1mf_Buffer, buffer_distance_or_field='8 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (10) (Buffer) (analysis)
    lagoas_1_2mf = f'{v_output}\lagoas_1_2mf'
    lagoas_1_2mf_Buffer = f'{v_output}\lagoas_1_2mf_Buffer'
    analysis.Buffer(in_features=lagoas_1_2mf, out_feature_class=lagoas_1_2mf_Buffer, buffer_distance_or_field='8 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (11) (Buffer) (analysis)
    lagoas_2_4mf = f'{v_output}\lagoas_2_4mf'
    lagoas_2_4mf_Buffer = f'{v_output}\lagoas_2_4mf_Buffer'
    analysis.Buffer(in_features=lagoas_2_4mf, out_feature_class=lagoas_2_4mf_Buffer, buffer_distance_or_field='8 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (12) (Buffer) (analysis)
    lagoas_maior_4mf = f'{v_output}\lagoas_maior_4mf'
    lagoas_maior_4mf_Buffer = f'{v_output}\lagoas_maior_4mf_Buffer'
    analysis.Buffer(in_features=lagoas_maior_4mf, out_feature_class=lagoas_maior_4mf_Buffer, buffer_distance_or_field='8 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Merge (Merge) (management)
    app_benesses_merge = rf'{v_output}\app_benesses_merge'
    management.Merge(inputs=[springs_benesses_buffer, wb_maior_4mf_Buffer, wb_2_4mf_Buffer, wb_1_2mf_Buffer, wb_0_1mf_Buffer, rivers_maior_4mf_Buffer, rivers_2_4mf_Buffer, rivers_1_2mf_Buffer, rivers_0_1mf_Buffer, lagoas_0_1mf_Buffer, lagoas_1_2mf_Buffer, lagoas_2_4mf_Buffer, lagoas_maior_4mf_Buffer], output=app_benesses_merge)

    # Process: Intersect (Intersect) (analysis)
    app_benesses_merge_Intersect = rf'{v_output}\app_benesses_merge_intersect'
    analysis.Intersect(in_features=[[app_benesses_merge, ''], [v_properties, '']], out_feature_class=app_benesses_merge_Intersect, join_attributes='ALL', cluster_tolerance='', output_type='INPUT')

    # Process: Dissolve (Dissolve) (management)
    app_benesses_merge_intersect_dis = rf'{v_output}\app_benesses_merge_intersect_dis'
    management.Dissolve(in_features=app_benesses_merge_Intersect, out_feature_class=app_benesses_merge_intersect_dis, dissolve_field=['INDEX'], statistics_fields=[], multi_part='MULTI_PART', unsplit_lines='DISSOLVE_LINES')

    # Process: Repair Geometry (Repair Geometry) (management)
    app_benesses_merge_intersect_dis_2 = management.RepairGeometry(in_features=app_benesses_merge_intersect_dis, delete_null='DELETE_NULL', validation_method='ESRI')[0]

    # Process: Erase (Erase) (analysis)
    app_benesses = rf'{v_output}\app_benesses'
    pol_erase = f'{v_output}\pol_erase'
    analysis.Erase(in_features=app_benesses_merge_intersect_dis_2, erase_features=pol_erase, out_feature_class=app_benesses, cluster_tolerance='')


In [ ]:
# funcao responsavel por criar os arquivos temporarios do modulo app_total do antigo Model Builder
def app_total(v_water_bodies, v_rivers, v_springs, v_input, v_output):
    # Process: Feature To Line (Feature To Line) (management)
    wb_process_app_toLine = f'{v_output}\wb_process_app_toLine'
    management.FeatureToLine(in_features=[v_water_bodies], out_feature_class=wb_process_app_toLine, cluster_tolerance='', attributes='ATTRIBUTES')

    # Process: Buffer (3) (Buffer) (analysis)
    wb_process_buffer_I = f'{v_output}\wb_process_buffer_I'
    analysis.Buffer(in_features=wb_process_app_toLine, out_feature_class=wb_process_buffer_I, buffer_distance_or_field='1 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Erase (Erase) (analysis)
    wb_process_max_width10 = f'{v_output}\wb_process_max_width10'
    analysis.Erase(in_features=v_water_bodies, erase_features=wb_process_buffer_I, out_feature_class=wb_process_max_width10, cluster_tolerance='')

    # Process: Buffer (6) (Buffer) (analysis)
    wb_process_max_10_buffer = f'{v_output}\wb_process_max_10_buffer'
    analysis.Buffer(in_features=wb_process_max_width10, out_feature_class=wb_process_max_10_buffer, buffer_distance_or_field='31 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Feature To Line (5) (Feature To Line) (management)
    wb_process_app_toLineII = f'{v_output}\wb_process_app_toLineII'
    management.FeatureToLine(in_features=[wb_process_max_width10], out_feature_class=wb_process_app_toLineII, cluster_tolerance='', attributes='ATTRIBUTES')

    # Process: Buffer (5) (Buffer) (analysis)
    wb_process_buffer_II = f'{v_output}\wb_process_buffer_II'
    analysis.Buffer(in_features=wb_process_app_toLineII, out_feature_class=wb_process_buffer_II, buffer_distance_or_field='4 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Erase (16) (Erase) (analysis)
    wb_process_max_width50 = f'{v_output}\wb_process_max_width50'
    analysis.Erase(in_features=wb_process_max_width10, erase_features=wb_process_buffer_II, out_feature_class=wb_process_max_width50, cluster_tolerance='')

    # Process: Buffer (8) (Buffer) (analysis)
    wb_process_max_50_buffer = f'{v_output}\wb_process_max_50_buffer'
    analysis.Buffer(in_features=wb_process_max_width50, out_feature_class=wb_process_max_50_buffer, buffer_distance_or_field='55 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Feature To Line (2) (Feature To Line) (management)
    wb_process_app_toLineIII = f'{v_output}\wb_process_app_toLineIII'
    management.FeatureToLine(in_features=[wb_process_max_width50], out_feature_class=wb_process_app_toLineIII, cluster_tolerance='', attributes='ATTRIBUTES')

    # Process: Buffer (7) (Buffer) (analysis)
    wb_process_buffer_III = f'{v_output}\wb_process_buffer_III'
    analysis.Buffer(in_features=wb_process_app_toLineIII, out_feature_class=wb_process_buffer_III, buffer_distance_or_field='20 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Erase (9) (Erase) (analysis)
    wb_process_max_width200 = f'{v_output}\wb_process_max_width200'
    analysis.Erase(in_features=wb_process_max_width50, erase_features=wb_process_buffer_III, out_feature_class=wb_process_max_width200, cluster_tolerance='')

    # Process: Buffer (11) (Buffer) (analysis)
    wb_process_max_200_buffer = f'{v_output}\wb_process_max_200_buffer'
    analysis.Buffer(in_features=wb_process_max_width200, out_feature_class=wb_process_max_200_buffer, buffer_distance_or_field='125 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Feature To Line (3) (Feature To Line) (management)
    wb_process_app_toLineIIII = f'{v_output}\wb_process_app_toLineIIII'
    management.FeatureToLine(in_features=[wb_process_max_width200], out_feature_class=wb_process_app_toLineIIII, cluster_tolerance='', attributes='ATTRIBUTES')

    # Process: Buffer (9) (Buffer) (analysis)
    wb_process_buffer_IIII = f'{v_output}\wb_process_buffer_IIII'
    analysis.Buffer(in_features=wb_process_app_toLineIIII, out_feature_class=wb_process_buffer_IIII, buffer_distance_or_field='75 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Erase (15) (Erase) (analysis)
    wb_process_max_width600 = f'{v_output}\wb_process_max_width600'
    analysis.Erase(in_features=wb_process_max_width200, erase_features=wb_process_buffer_IIII, out_feature_class=wb_process_max_width600, cluster_tolerance='')

    # Process: Buffer (12) (Buffer) (analysis)
    wb_process_max_600_buffer = f'{v_output}\wb_process_max_600_buffer'
    analysis.Buffer(in_features=wb_process_max_width600, out_feature_class=wb_process_max_600_buffer, buffer_distance_or_field='300 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Feature To Line (4) (Feature To Line) (management)
    wb_process_app_toLineIIIII = f'{v_output}\wb_process_app_toLineIIIII'
    management.FeatureToLine(in_features=[wb_process_max_width600], out_feature_class=wb_process_app_toLineIIIII, cluster_tolerance='', attributes='ATTRIBUTES')

    # Process: Buffer (10) (Buffer) (analysis)
    wb_process_buffer_IIIII = f'{v_output}\wb_process_buffer_IIIII'
    analysis.Buffer(in_features=wb_process_app_toLineIIIII, out_feature_class=wb_process_buffer_IIIII, buffer_distance_or_field='75 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Erase (10) (Erase) (analysis)
    wb_process_max_width_maior_600 = f'{v_output}\wb_process_max_width_maior_600'
    analysis.Erase(in_features=wb_process_max_width600, erase_features=wb_process_buffer_IIIII, out_feature_class=wb_process_max_width_maior_600, cluster_tolerance='')

    # Process: Buffer (13) (Buffer) (analysis)
    wb_process_max_maior_600_buffer = f'{v_output}\wb_process_max_maior_600_buffer'
    analysis.Buffer(in_features=wb_process_max_width_maior_600, out_feature_class=wb_process_max_maior_600_buffer, buffer_distance_or_field='675 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Merge (Merge) (management)
    app_total_wb = rf'{v_output}\app_total_wb'
    management.Merge(inputs=[wb_process_max_10_buffer, wb_process_max_50_buffer, wb_process_max_200_buffer, wb_process_max_600_buffer, wb_process_max_maior_600_buffer], output=app_total_wb)

    # Process: Buffer (4) (Buffer) (analysis)
    app_total_springs = rf'{v_output}\app_total_springs'
    analysis.Buffer(in_features=v_springs, out_feature_class=app_total_springs, buffer_distance_or_field='50 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (2) (Buffer) (analysis)
    lagoa_area_maior_20 = f'{v_output}\lagoa_area_maior_20'
    app_total_lagoa_maior_20 = rf'{v_output}\app_total_lagoa_maior_20'
    analysis.Buffer(in_features=lagoa_area_maior_20, out_feature_class=app_total_lagoa_maior_20, buffer_distance_or_field='100 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (Buffer) (analysis)
    lagoa_area_1_20 = f'{v_output}\lagoa_area_1_20'
    app_total_lagoa_1_20 = rf'{v_output}\app_total_lagoa_1_20'
    analysis.Buffer(in_features=lagoa_area_1_20, out_feature_class=app_total_lagoa_1_20, buffer_distance_or_field='50 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Buffer (14) (Buffer) (analysis)
    app_total_rivers_2 = rf'{v_output}\app_total_rivers'
    analysis.Buffer(in_features=v_rivers, out_feature_class=app_total_rivers_2, buffer_distance_or_field='30 Meters', line_side='FULL', line_end_type='ROUND', dissolve_option='NONE', dissolve_field=[], method='PLANAR')

    # Process: Merge (2) (Merge) (management) 
    app_total_Merge = rf'{v_output}\app_total_Merge'
    v_ppa_areas = f'{v_input}\ppa_areas'
    
    if input_ppa_areas == "":       
        management.Merge(inputs=[app_total_wb, app_total_springs, app_total_lagoa_maior_20, app_total_lagoa_1_20, app_total_rivers_2], output=app_total_Merge)
    else:     
        management.Merge(inputs=[app_total_wb, app_total_springs, app_total_lagoa_maior_20, app_total_lagoa_1_20, app_total_rivers_2, v_ppa_areas], output=app_total_Merge)
        
    # Process: Dissolve (Dissolve) (management)
    app_total_Erase_Dissolve = rf'{v_output}\app_total_Erase_Dissolve'
    management.Dissolve(in_features=app_total_Merge, out_feature_class=app_total_Erase_Dissolve, dissolve_field=[], statistics_fields=[], multi_part='SINGLE_PART', unsplit_lines='DISSOLVE_LINES')

    # Process: Erase (2) (Erase) (analysis)
    pol_erase = f'{v_output}\pol_erase'
    app_total = rf'{v_output}\app_total'
    analysis.Erase(in_features=app_total_Erase_Dissolve, erase_features=pol_erase, out_feature_class=app_total, cluster_tolerance='')


In [ ]:
# funcao responsavel por criar os arquivos temporarios do modulo passivos_ativos do antigo Model Builder
def passivos_ativos(v_properties, v_veg_2008, v_veg_current, v_intermediate_results, v_intermediate_results_outputs, v_outputs):
    # Process: Intersect (3) (Intersect) (analysis)
    properties_Intersect = f'{v_intermediate_results_outputs}\properties_Intersect'
    analysis.Intersect(in_features=[[v_properties, ""], [v_veg_current, ""]], out_feature_class=properties_Intersect, join_attributes="ALL", cluster_tolerance="", output_type="INPUT")

    # Process: Dissolve (3) (Dissolve) (management)
    vegetation_imovel_current = rf'{v_outputs}\vegetation_imovel_current'
    management.Dissolve(in_features=properties_Intersect, out_feature_class=vegetation_imovel_current, dissolve_field=["INDEX"], statistics_fields=[], multi_part="MULTI_PART", unsplit_lines="DISSOLVE_LINES")

    # Process: Erase (2) (Erase) (analysis)
    app_total = rf'{v_intermediate_results}\app_total'
    rl_current = rf'{v_outputs}\rl_current'
    analysis.Erase(in_features=vegetation_imovel_current, erase_features=app_total, out_feature_class=rl_current, cluster_tolerance="")

    # Process: Intersect (2) (Intersect) (analysis)
    veg_2008_Intersect = rf'{v_intermediate_results_outputs}\veg_2008_Intersect'
    analysis.Intersect(in_features=[[v_veg_2008, ""], [v_properties, ""]], out_feature_class=veg_2008_Intersect, join_attributes="ALL", cluster_tolerance="", output_type="INPUT")

    # Process: Dissolve (2) (Dissolve) (management)
    vegetation_imovel_2008 =  rf'{v_outputs}\vegetation_imovel_2008'
    management.Dissolve(in_features=veg_2008_Intersect, out_feature_class=vegetation_imovel_2008, dissolve_field=["INDEX"], statistics_fields=[], multi_part="MULTI_PART", unsplit_lines="DISSOLVE_LINES")

    # Process: Erase (3) (Erase) (analysis)
    rl_2008 = rf'{v_outputs}\rl_2008'
    analysis.Erase(in_features=vegetation_imovel_2008, erase_features=app_total, out_feature_class=rl_2008, cluster_tolerance="")

    # Process: Erase (Erase) (analysis)
    df_pos_2008 = f'{v_outputs}\df_pos_2008'
    analysis.Erase(in_features=vegetation_imovel_2008, erase_features=vegetation_imovel_current, out_feature_class=df_pos_2008, cluster_tolerance="")

    # Process: Intersect (4) (Intersect) (analysis)
    app_total_inter = rf'{v_intermediate_results_outputs}\app_total_inter'
    analysis.Intersect(in_features=[[app_total, ""], [v_properties, ""]], out_feature_class=app_total_inter, join_attributes="ALL", cluster_tolerance="", output_type="INPUT")

    # Process: Dissolve (4) (Dissolve) (management)
    app_total_imovel = rf'{v_outputs}\app_total_imovel'
    management.Dissolve(in_features=app_total_inter, out_feature_class=app_total_imovel, dissolve_field=["INDEX"], statistics_fields=[], multi_part="MULTI_PART", unsplit_lines="DISSOLVE_LINES")

    # Process: Erase (6) (Erase) (analysis)
    pasrest = f'{v_outputs}\pasrest'
    analysis.Erase(in_features=df_pos_2008, erase_features=app_total_imovel, out_feature_class=pasrest, cluster_tolerance="")

    # Process: Clip (Clip) (analysis)
    apprf = rf'{v_outputs}\apprf'
    analysis.Clip(in_features=vegetation_imovel_current, clip_features=app_total_imovel, out_feature_class=apprf, cluster_tolerance="")

    # Process: Erase (5) (Erase) (analysis)
    app_benesses = rf'{v_intermediate_results}\app_benesses'
    app_benesses_Erase = rf'{v_intermediate_results_outputs}\app_benesses_Erase'
    analysis.Erase(in_features=app_benesses, erase_features=apprf, out_feature_class=app_benesses_Erase, cluster_tolerance="")

    # Process: Erase (4) (Erase) (analysis)
    app_total_imovel_Erase = rf'{v_intermediate_results_outputs}\app_total_imovel_Erase'
    analysis.Erase(in_features=app_total_imovel, erase_features=apprf, out_feature_class=app_total_imovel_Erase, cluster_tolerance="")

    # Process: Clip (2) (Clip) (analysis)
    df_pos_2008_app = f'{v_outputs}\df_pos_2008_app'
    analysis.Clip(in_features=df_pos_2008, clip_features=app_total_imovel_Erase, out_feature_class=df_pos_2008_app, cluster_tolerance="")

    # Process: Merge (Merge) (management)
    appd_merge = rf'{v_intermediate_results_outputs}\appd_merge'
    management.Merge(inputs=[app_benesses_Erase, df_pos_2008_app], output=appd_merge)

    # Process: Dissolve (Dissolve) (management)
    appd_merge_Dissolve1 = rf'{v_intermediate_results_outputs}\appd_merge_Dissolve1'
    management.Dissolve(in_features=appd_merge, out_feature_class=appd_merge_Dissolve1, dissolve_field=["INDEX"], statistics_fields=[], multi_part="SINGLE_PART", unsplit_lines="DISSOLVE_LINES")

    # Process: Intersect (Intersect) (analysis)
    appd_int_dis = rf'{v_intermediate_results_outputs}\appd_int_dis'
    analysis.Intersect(in_features=[[v_properties, ""], [appd_merge_Dissolve1, ""]], out_feature_class=appd_int_dis, join_attributes="ALL", cluster_tolerance="", output_type="INPUT")

    # Process: Dissolve (5) (Dissolve) (management)
    appd = rf'{v_outputs}\appd'
    management.Dissolve(in_features=appd_int_dis, out_feature_class=appd, dissolve_field=["INDEX"], statistics_fields=[], multi_part="MULTI_PART", unsplit_lines="DISSOLVE_LINES")


In [ ]:
# funcao responsavel por criar os arquivos temporarios do modulo calculate do antigo Model Builder alem dos dados finais de propriedades
def calculate(v_properties, v_outputs, v_outputs_calculate):
    # Process: Copy Features (Copy Features) (management)
    properties_join = f'{v_outputs_calculate}\properties_join'
    management.CopyFeatures(in_features=v_properties, out_feature_class=properties_join, config_keyword="", spatial_grid_1=0, spatial_grid_2=0, spatial_grid_3=0)

    rl_current = rf'{v_outputs}\rl_current'
    # Process: Add Field (Add Field) (management)
    rl_current_4 = management.AddField(in_table=rl_current, field_name="rlCurrent", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (Calculate Field) (management)
    rl_current_3 = management.CalculateField(in_table=rl_current_4, field="rlCurrent", expression="!Shape_Area!/10000", expression_type="PYTHON", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Join Field (Join Field) (management)
    properties_join_11 = management.JoinField(in_data=properties_join, in_field="INDEX", join_table=rl_current_3, join_field="INDEX", fields=["rlCurrent"])[0]

    # Process: Add Field (2) (Add Field) (management)
    rl_2008 = rf'{v_outputs}\rl_2008'
    rl_2008_2 = management.AddField(in_table=rl_2008, field_name="rlmarco", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (2) (Calculate Field) (management)
    rl_2008_4 = management.CalculateField(in_table=rl_2008_2, field="rlmarco", expression="!Shape_Area!/10000", expression_type="PYTHON", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Join Field (2) (Join Field) (management)
    properties_join_3 = management.JoinField(in_data=properties_join_11, in_field="INDEX", join_table=rl_2008_4, join_field="INDEX", fields=["rlmarco"])[0]

    # Process: Add Field (3) (Add Field) (management)
    pasrest = f'{v_outputs}\pasrest'
    pasrest_2 = management.AddField(in_table=pasrest, field_name="rlrest", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (3) (Calculate Field) (management)
    pasrest_4 = management.CalculateField(in_table=pasrest_2, field="rlrest", expression="!Shape_Area!/10000", expression_type="PYTHON", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Join Field (3) (Join Field) (management)
    properties_join_13 = management.JoinField(in_data=properties_join_3, in_field="INDEX", join_table=pasrest_4, join_field="INDEX", fields=["rlrest"])[0]

    # Process: Add Field (4) (Add Field) (management)
    df_pos_2008 = f'{v_outputs}\df_pos_2008'
    df_pos_2008_2 = management.AddField(in_table=df_pos_2008, field_name="dfpos08", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (4) (Calculate Field) (management)
    df_pos_2008_4 = management.CalculateField(in_table=df_pos_2008_2, field="dfpos08", expression="!Shape_Area!/10000", expression_type="PYTHON", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Join Field (4) (Join Field) (management)
    properties_join_14 = management.JoinField(in_data=properties_join_13, in_field="INDEX", join_table=df_pos_2008_4, join_field="INDEX", fields=["dfpos08"])[0]

    # Process: Add Field (5) (Add Field) (management)
    app_total_imovel = rf'{v_outputs}\app_total_imovel'
    app_total_imovel_2 = management.AddField(in_table=app_total_imovel, field_name="apptotal", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (5) (Calculate Field) (management)
    app_total_imovel_4 = management.CalculateField(in_table=app_total_imovel_2, field="apptotal", expression="!Shape_Area!/10000", expression_type="PYTHON", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Join Field (5) (Join Field) (management)
    properties_join_15 = management.JoinField(in_data=properties_join_14, in_field="INDEX", join_table=app_total_imovel_4, join_field="INDEX", fields=["apptotal"])[0]

    # Process: Add Field (6) (Add Field) (management)
    appd = rf'{v_outputs}\appd'
    appd_2 = management.AddField(in_table=appd, field_name="appd", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (6) (Calculate Field) (management)
    appd_4 = management.CalculateField(in_table=appd_2, field="appd", expression="!Shape_Area!/10000", expression_type="PYTHON", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Join Field (6) (Join Field) (management)
    properties_join_16 = management.JoinField(in_data=properties_join_15, in_field="INDEX", join_table=appd_4, join_field="INDEX", fields=["appd"])[0]

    # Process: Add Field (7) (Add Field) (management)
    apprf = rf'{v_outputs}\apprf'
    apprf_2 = management.AddField(in_table=apprf, field_name="apprf", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (7) (Calculate Field) (management)
    apprf_4 = management.CalculateField(in_table=apprf_2, field="apprf", expression="!Shape_Area!/10000", expression_type="PYTHON", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Join Field (7) (Join Field) (management)
    properties_join_17 = management.JoinField(in_data=properties_join_16, in_field="INDEX", join_table=apprf_4, join_field="INDEX", fields=["apprf"])[0]

    # Process: Add Field (9) (Add Field) (management)
    vegetation_imovel_2008 =  rf'{v_outputs}\vegetation_imovel_2008'
    vegetation_imovel_2008_3 = management.AddField(in_table=vegetation_imovel_2008, field_name="vegmarco", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (9) (Calculate Field) (management)
    vegetation_imovel_2008_4 = management.CalculateField(in_table=vegetation_imovel_2008_3, field="vegmarco", expression="!Shape_Area!/10000", expression_type="PYTHON", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Join Field (8) (Join Field) (management)
    properties_join_18 = management.JoinField(in_data=properties_join_17, in_field="INDEX", join_table=vegetation_imovel_2008_4, join_field="INDEX", fields=["vegmarco"])[0]

    # Process: Add Field (10) (Add Field) (management)
    vegetation_imovel_current = rf'{v_outputs}\vegetation_imovel_current'
    vegetation_imovel_current_3 = management.AddField(in_table=vegetation_imovel_current, field_name="vegcurrent", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (10) (Calculate Field) (management)
    vegetation_imovel_current_4 = management.CalculateField(in_table=vegetation_imovel_current_3, field="vegcurrent", expression="!Shape_Area!/10000", expression_type="PYTHON", code_block="", field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Join Field (9) (Join Field) (management)
    properties_join_19 = management.JoinField(in_data=properties_join_18, in_field="INDEX", join_table=vegetation_imovel_current_4, join_field="INDEX", fields=["vegcurrent"])[0]
    
    # Process: Calculate Field (11) (Calculate Field) (management)
    cb_properties_join4 = """def findNull(field):
        if field == None:
            return 0
        else:
            return field
    """
    properties_join_4 = management.CalculateField(in_table=properties_join_19, field="rlrest", expression="findNull(!rlrest!)", expression_type="PYTHON", code_block=cb_properties_join4, field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Calculate Field (12) (Calculate Field) (management)
    cb_properties_join_7 = """def findNull(field):
                                if field == None:
                                    return 0
                                else:
                                    return field
                            """
    properties_join_7 = management.CalculateField(in_table=properties_join_4, field="vegmarco", expression="findNull( !vegmarco! )", expression_type="PYTHON", code_block=cb_properties_join_7, field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Calculate Field (13) (Calculate Field) (management)
    cb_properties_join_6 = """def findNull(field):
                                if field == None:
                                    return 0
                                else:
                                    return field
                            """
    properties_join_6 = management.CalculateField(in_table=properties_join_7, field="vegcurrent", expression="findNull( !vegcurrent! )", expression_type="PYTHON", code_block=cb_properties_join_6, field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Add Field (8) (Add Field) (management)
    properties_join_8 = management.AddField(in_table=properties_join_6, field_name="pascomp", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Add Field (11) (Add Field) (management)
    properties_join_5 = management.AddField(in_table=properties_join_8, field_name="rlexigida_p", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (14) (Calculate Field) (management)
    cb_properties_join_20 = """def getReservaExigida(amzlegal, area, vegmarco, vegcurrent, mf, vegetacao, zee):
                                vegmarco_p = vegmarco*100/area
                                vegcurrent_p = vegcurrent*100/area
                                rlexigida = 0

                                if amzlegal == 0:
                                    rlexigida = 20
                                    if (vegcurrent_p < rlexigida) & (mf <= 4):
                                        rlexigida = rlexigida 
                                        if vegmarco_p < rlexigida: 
                                            rlexigida = vegmarco_p          

                                elif (amzlegal == 1) & (vegetacao == 'floresta'):
                                    if zee == 1:
                                        rlexigida = 50
                                        if (vegcurrent_p < rlexigida) & (mf <= 4):
                                            rlexigida = rlexigida
                                            if vegmarco_p < rlexigida:
                                                rlexigida = vegmarco_p            
                                    elif zee == 0:
                                        rlexigida = 80
                                        if (vegcurrent_p < rlexigida) & (mf <= 4):
                                            rlexigida = rlexigida
                                            if vegmarco_p < rlexigida:     
                                                rlexigida = vegmarco_p    

                                elif (amzlegal == 1) & (vegetacao == 'cerrado'):
                                    rlexigida = 35
                                    if (vegcurrent_p < rlexigida) & (mf <= 4):
                                        rlexigida = rlexigida
                                        if vegmarco_p < rlexigida:
                                            rlexigida = vegmarco_p

                                elif (amzlegal == 1) & (vegetacao == 'campos gerais'):
                                    rlexigida = 20
                                    if (vegcurrent_p < rlexigida) & (mf <= 4):
                                        rlexigida = rlexigida
                                        if vegmarco_p < rlexigida:
                                            rlexigida = vegmarco_p

                                return rlexigida
                            """
    properties_join_20 = management.CalculateField(in_table=properties_join_5, field="rlexigida_p", expression="getReservaExigida( !amzlegal!, !area!, !vegmarco!, !vegcurrent!, !mf!, !fito!, !zee! )", expression_type="PYTHON", code_block=cb_properties_join_20, field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]

    # Process: Calculate Field (8) (Calculate Field) (management)
    cb_properties_join_12 = """def getPassivoCompensacao(reserva_exigida, mf, vegetacaomarco, vegetacaocurrent, area, reserva_restauro):
                                vegetacaomarco_p = (vegetacaomarco*100)/area
                                vegetacaocurrent_p = (vegetacaocurrent*100)/area
                                reserva_restauro_p =(reserva_restauro*100)/area
                                passivo_comp = 0
                                if (mf > 4):
                                    if (vegetacaocurrent_p < reserva_exigida):
                                        diff = reserva_exigida - vegetacaocurrent_p
                                        if (diff - reserva_restauro_p) > 0:
                                            passivo_comp = diff - reserva_restauro_p
                                            passivo_comp = passivo_comp/100*area

                                return passivo_comp
                            """
    properties_join_12 = management.CalculateField(in_table=properties_join_20, field="pascomp", expression="getPassivoCompensacao( !rlexigida_p! , !mf!, !vegmarco!, !vegcurrent!, !area!, !rlrest!)", expression_type="PYTHON", code_block=cb_properties_join_12, field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]
    
    # Process: Add Field (12) (Add Field) (management)
    properties_join_10 = management.AddField(in_table=properties_join_12, field_name="reservativo", field_type="DOUBLE", field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]

    # Process: Calculate Field (15) (Calculate Field) (management)
    cb_properties_join_9 = """def getAtivo(reserva_comp, reserva_exigida, amz_legal, vegetacao, vegetacao_current, zee, area):
                                reserva_ativo = 0
                                vegetacao_current_p = vegetacao_current*100/area

                                if reserva_comp == 0:
                                    if amz_legal == 1:
                                        if vegetacao == 'floresta':
                                            if zee == 1:
                                                if vegetacao_current_p > 50:
                                                    reserva_ativo_p = vegetacao_current_p - 50
                                                    reserva_ativo = reserva_ativo_p/100*area

                                            elif zee == 0:
                                                if vegetacao_current_p > 80:
                                                    reserva_ativo_p = vegetacao_current_p - 80
                                                    reserva_ativo = reserva_ativo_p/100*area

                                        elif vegetacao == 'cerrado':
                                            if vegetacao_current_p > 35:
                                                reserva_ativo_p = vegetacao_current_p - 35
                                                reserva_ativo = reserva_ativo_p/100*area

                                        elif vegetacao == 'campos gerais':
                                            if vegetacao_current_p > 20:
                                                reserva_ativo_p = vegetacao_current_p - 20
                                                reserva_ativo = reserva_ativo_p/100*area

                                    if amz_legal == 0:
                                        if vegetacao_current_p > 20:
                                            reserva_ativo_p = vegetacao_current_p - 20
                                            reserva_ativo = reserva_ativo_p/100*area
                                return reserva_ativo
                            """
    properties_join_9 = management.CalculateField(in_table=properties_join_10, field="reservativo", expression="getAtivo( !pascomp!, !rlexigida_p!, !amzlegal!,!fito!, !vegcurrent!, !zee!, !area!)", expression_type="PYTHON", code_block=cb_properties_join_9, field_type="TEXT", enforce_domains="NO_ENFORCE_DOMAINS")[0]
    
    return properties_join_9


In [ ]:
def properties(v_lakes, v_veg_current, v_saf, v_others_uses, v_properties_join, v_output, v_input):
    
    if input_saf == "":
        management.CreateFeatureclass(out_path=v_input, out_name='saf', spatial_reference=v_lakes)
        management.AddField(in_table=v_saf, field_name='ID_PROPRIE', field_type='DOUBLE', field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]     
    
    if input_others_uses == "":       
        management.CreateFeatureclass(out_path=v_input, out_name='others_uses', spatial_reference=v_lakes)
        management.AddField(in_table=v_others_uses, field_name='ID_PROPRIE', field_type='DOUBLE', field_precision=None, field_scale=None, field_length=None, field_alias="", field_is_nullable="NULLABLE", field_is_required="NON_REQUIRED", field_domain="")[0]   

    # Process: Merge (Merge) (management)
    uso_solo_merge = rf'{v_output}\uso_solo_merge'
    management.Merge(inputs=[v_lakes, v_veg_current, v_saf, v_others_uses], output=uso_solo_merge)
    
    # Process: Erase (Erase) (analysis)
    pastagem = rf'{v_output}\pastagem'
    analysis.Erase(in_features=v_properties_join, erase_features=uso_solo_merge, out_feature_class=pastagem, cluster_tolerance='')
    
    # Process: Calculate Geometry Attributes (management)
    management.CalculateGeometryAttributes(in_features=pastagem, geometry_property='pastagem AREA', area_unit='HECTARES', coordinate_system=v_lakes)
    management.CalculateGeometryAttributes(in_features=v_saf, geometry_property='saf AREA', area_unit='HECTARES', coordinate_system=v_lakes)
    management.CalculateGeometryAttributes(in_features=v_others_uses, geometry_property='outros_usos AREA', area_unit='HECTARES', coordinate_system=v_lakes)
    
    # Process: Dissolve (management)
    saf_dissolve = rf'{v_output}\saf_dissolve'
    management.Dissolve(in_features=v_saf, out_feature_class=saf_dissolve, dissolve_field='ID_PROPRIE', statistics_fields='saf SUM', multi_part='MULTI_PART', unsplit_lines='DISSOLVE_LINES')
    management.AlterField(in_table=saf_dissolve, field='SUM_saf', new_field_name='saf', new_field_alias='saf', field_type='DOUBLE', field_length=8, field_is_nullable='NULLABLE')
    
    others_uses_dissolve = rf'{v_output}\outros_usos_dissolve'
    management.Dissolve(in_features=v_others_uses, out_feature_class=others_uses_dissolve, dissolve_field='ID_PROPRIE', statistics_fields='OUTROS_USOS SUM', multi_part='MULTI_PART', unsplit_lines='DISSOLVE_LINES')
    management.AlterField(in_table=others_uses_dissolve, field='SUM_outros_usos', new_field_name='outros_usos', new_field_alias='outros_usos', field_type='DOUBLE', field_length=8, field_is_nullable='NULLABLE')
    
    # Process: Join Field (Join Field) (management)
    saf_join = management.JoinField(in_data=v_properties_join, in_field='ID_IMOVEL', join_table=saf_dissolve, join_field='ID_PROPRIE', fields=['saf'])
    others_uses_join = management.JoinField(in_data=v_properties_join, in_field='ID_IMOVEL', join_table=others_uses_dissolve, join_field='ID_PROPRIE', fields=['outros_usos'])
    pastagem_join = management.JoinField(in_data=v_properties_join, in_field='ID_IMOVEL', join_table=pastagem, join_field='ID_IMOVEL', fields=['pastagem'])


In [ ]:
# funcao responsavel por copiar o APRX padrao para a pasta de output e atualizar os datasources
def aprx_project(inp_toolbox_path, output, fgdb_path_new):
    aprx_path = os.path.join(inp_toolbox_path,'Project', 'layout_map_series.aprx')
    fgdb_path = os.path.join(inp_toolbox_path,'Project', 'forest_code_analisys.gdb')
    aprx_path_new = os.path.join(output,'project_map_series.aprx')
    aprx = arcpy.mp.ArcGISProject(aprx_path)
    aprx.updateConnectionProperties(fgdb_path, fgdb_path_new)
    aprx.saveACopy(aprx_path_new)
    print(aprx_path_new)
    return aprx_path_new

In [ ]:
# funcao responsavel por imprimir o maps series
def print_map_series(aprx, output):
    
    output_jpeg = os.path.join(output, 'mapas_jpeg')
    if not os.path.exists(output_jpeg):
        os.mkdir(output_jpeg)
    
    output_pdf = os.path.join(output, 'mapas_pdf')
    if not os.path.exists(output_pdf):
        os.mkdir(output_pdf)
        
    project = arcpy.mp.ArcGISProject(aprx)  # aprx é o arquivo onde está configurado o Map Series
    lyt = project.listLayouts()[0] #x: o numero do layout do projeto
    if not lyt.mapSeries is None:
        ms = lyt.mapSeries
        if ms.enabled:
            for pageNum in range(1, ms.pageCount + 1):
                ms.currentPageNumber = pageNum
                pageName = f'{int(ms.pageRow.ID_IMOVEL)} - {ms.pageRow.NM_PRODTOR}' # NM_PRODTOR é o campo da tabela de atributos que representa o nome do produtor
                lyt.exportToJPEG(os.path.join(output_jpeg, f'{pageName}.jpg'), resolution=300)
                lyt.exportToPDF(os.path.join(output_pdf, f'{pageName}.pdf'), resolution=300)

In [ ]:
dataset_list = list(create_fgdb(output))

In [ ]:
module_input = add_input(input_lakes, input_water_bodies, input_rivers, input_properties, input_springs, input_veg_2008, input_veg_current, input_ppa_areas, input_saf, input_others_uses, dataset_list[0], input_toolbox_path)

In [ ]:
module_config = config(module_input[0], module_input[1], module_input[2], module_input[3], dataset_list[1])

In [ ]:
module_app_consolidada = app_consolidada(module_input[3], module_input[4], dataset_list[1])

In [ ]:
module_app_total = app_total(module_input[1], module_input[2], module_input[4], dataset_list[0], dataset_list[1])

In [ ]:
module_passivos_ativos = passivos_ativos(module_input[3], module_input[5], module_input[6], dataset_list[1], dataset_list[2], dataset_list[3])

In [ ]:
module_calculate = calculate(module_input[3], dataset_list[3], dataset_list[4])

In [ ]:
module_properties = properties(module_input[0], module_input[6], module_input[8], module_input[9], module_calculate, dataset_list[3], dataset_list[0])

In [ ]:
module_aprx_project = aprx_project(input_toolbox_path, output, dataset_list[5])

In [ ]:
module_print_map_series = print_map_series(module_aprx_project, output)